In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/notebooks/fabian/randomforestsolspipeline.pkl"
pipeline = joblib.load(pipeline_path)

In [3]:
# Charger le CSV
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data/datatest/testpredict1/sols1.csv"
df_test = pd.read_csv(csv_path, delimiter=';')

# Nettoyer les noms de colonnes comme à l'entraînement
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

df_test.columns = [clean_col(c) for c in df_test.columns]

In [4]:
df_test

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,tableau_1
Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,Murs en intersection,Murs coupés (u),Murs coupés (Ids),Murs coupants (u),Murs coupants (Ids),Poutres en intersection,Poutres coupés (u),Poutres coupés (Ids),Poutres coupants (u),Poutres coupants (Ids),Poteaux en intersection,Poteaux coupés (u),Poteaux coupés (Ids),Poteaux coupants (u),Poteaux coupants (Ids),Ouvertures,Sol multicouche,Profil modifié,Image,Catégorie,Type prédéfini d'IFC,Exporter au format IFC sous,Exporter au format IFC,IfcGUID,A une association,Enrobage d'armature - Face inférieure,Enrobage d'armature - Face supérieure,Enrobage d'armature - Autres faces,Variantes,Volume,Surface,Phase de démolition,Phase de création,Commentaires,Inclinaison,Niveau,Famille et type,Famille,Type,Nom de la famille,Nom du type,ID du type,Structure,Périmètre,Décalage par rapport au niveau,Epaisseur,Lié au volume,Etude de l'élévation à la base,Etude de l'élévation en haut,Epaisseur du porteur,Elévation au niveau du noyau inférieur,Elévation au niveau du noyau supérieur,Elévation en haut,Elévation à la base,Identifiant,Limite de pièce,Visible dans les nomenclatures,Légende détails,Composition,Finition,Niveau Fini,IdGtcParameter,IdGtcParameterCreationTime,Image du type,Note d'identification,Type: Type prédéfini d'IFC,Exporter le type au format IFC sous,Exporter le type au format IFC,Type IfcGUID,Modèle,Fabricant,Commentaires du type,URL,Description,Matériau structurel,Rugosité,Coefficient d'absorbance,Masse thermique,Résistance thermique (R),Coefficient de transfert thermique (U),Description de l'assemblage,Code d'assemblage,Couleur vue détail faible,Motif vue détail faible,Epaisseur par défaut,Marque de type,Coût,Fonction,T - Lot,T - Teinte,T - Référence,T - Fabricant,T - Finition,DAP épaisseur,DAP - CF,DAP - DDC épaisseur,Teinte,Reférence
636513,NaN,NaN,NaN,NaN,EIF_Sol - 400 mm,0,4,"636680,636681,636682,636683",0,NaN,0,23,"1235338,1235484,1235557,1235604,1235645,1235841,1235970,1236021,1236042,1236098,1236128,1236166,1236283,1236457,1236784,1236885,1236951,1236987,1237092,1237151,1237486,1238049,1248419",0,NaN,0,27,"636523,636551,636553,636575,636577,636579,636581,636583,636585,636587,636627,636706,636718,636720,636722,636724,636726,636728,639691,639694,639700,639703,639705,639709,639711,639750,639753",0,NaN,0,False,False,<Aucun>,Sols,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNPQ,0,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,-1,"535,887167864421","1339,71791966105",Aucun(e),Nouvelle construction,NaN,0,A&B_SS1,Sol: EIF_Sol - 400 mm,Sol,EIF_Sol - 400 mm,NaN,NaN,1406624,1,"162,177223428329","0,31","0,4",0,"134,96","135,36","0,4","-3,34000000000023","-2,94000000000023","-2,94000000000023","-3,34000000000023",NaN,1,0,NaN,NaN,NaN,0,0,0,<Aucun>,NaN,NaN,NaN,0,0UcXdq41b0rv1UZqd9YDoK,NaN,NaN,NaN,NaN,NaN,EIF_STR - BETON,3,"0,7",604440,"0,382409177820268","2,615",NaN,NaN,0,NaN,"0,4",NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
636929,NaN,NaN,NaN,NaN,EIF_Sol - 230 mm,3,7,"636502,636506,636846,636857,639749,639756,639759",0,NaN,5,23,"1107656,1108070,1108469,1111200,1111442,1111535,1113864,1114165,1115604,1116181,1116183,1118803,1118912,1118966,1129058,1138314,1141144,1141591,1143089,1180899,1180969,1181061,1181234",0,NaN,0,0,NaN,0,NaN,0,False,False,<Aucun>,Sols,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNWw,0,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,-1,"107,922940125838","469,230174460166",Aucun(e),Nouvelle construction,NaN,0,C_RDC,Sol: EIF_Sol - 230 mm,Sol,EIF_Sol - 230 mm,NaN,NaN,684287,1,"104,846492013679","-0,13","0,23",0,"138,42","138,65","0,23","0,119999999999752","0,349999999999752","0,349999999999752","0,119999999999752",NaN,1,0,NaN,NaN,NaN,0,0,0,<Aucun>,NaN,NaN,NaN,0,0gcOVK$Xn1XgDKvIBnA8vn,NaN,NaN,NaN,NaN,NaN,EIF_STR - BETON,3,"0,7",347553,"0,219885277246654","4,54782608695652",NaN,NaN,0,NaN,"0

In [5]:
df_test

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,tableau_1
Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,Murs en intersection,Murs coupés (u),Murs coupés (Ids),Murs coupants (u),Murs coupants (Ids),Poutres en intersection,Poutres coupés (u),Poutres coupés (Ids),Poutres coupants (u),Poutres coupants (Ids),Poteaux en intersection,Poteaux coupés (u),Poteaux coupés (Ids),Poteaux coupants (u),Poteaux coupants (Ids),Ouvertures,Sol multicouche,Profil modifié,Image,Catégorie,Type prédéfini d'IFC,Exporter au format IFC sous,Exporter au format IFC,IfcGUID,A une association,Enrobage d'armature - Face inférieure,Enrobage d'armature - Face supérieure,Enrobage d'armature - Autres faces,Variantes,Volume,Surface,Phase de démolition,Phase de création,Commentaires,Inclinaison,Niveau,Famille et type,Famille,Type,Nom de la famille,Nom du type,ID du type,Structure,Périmètre,Décalage par rapport au niveau,Epaisseur,Lié au volume,Etude de l'élévation à la base,Etude de l'élévation en haut,Epaisseur du porteur,Elévation au niveau du noyau inférieur,Elévation au niveau du noyau supérieur,Elévation en haut,Elévation à la base,Identifiant,Limite de pièce,Visible dans les nomenclatures,Légende détails,Composition,Finition,Niveau Fini,IdGtcParameter,IdGtcParameterCreationTime,Image du type,Note d'identification,Type: Type prédéfini d'IFC,Exporter le type au format IFC sous,Exporter le type au format IFC,Type IfcGUID,Modèle,Fabricant,Commentaires du type,URL,Description,Matériau structurel,Rugosité,Coefficient d'absorbance,Masse thermique,Résistance thermique (R),Coefficient de transfert thermique (U),Description de l'assemblage,Code d'assemblage,Couleur vue détail faible,Motif vue détail faible,Epaisseur par défaut,Marque de type,Coût,Fonction,T - Lot,T - Teinte,T - Référence,T - Fabricant,T - Finition,DAP épaisseur,DAP - CF,DAP - DDC épaisseur,Teinte,Reférence
636513,NaN,NaN,NaN,NaN,EIF_Sol - 400 mm,0,4,"636680,636681,636682,636683",0,NaN,0,23,"1235338,1235484,1235557,1235604,1235645,1235841,1235970,1236021,1236042,1236098,1236128,1236166,1236283,1236457,1236784,1236885,1236951,1236987,1237092,1237151,1237486,1238049,1248419",0,NaN,0,27,"636523,636551,636553,636575,636577,636579,636581,636583,636585,636587,636627,636706,636718,636720,636722,636724,636726,636728,639691,639694,639700,639703,639705,639709,639711,639750,639753",0,NaN,0,False,False,<Aucun>,Sols,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNPQ,0,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,-1,"535,887167864421","1339,71791966105",Aucun(e),Nouvelle construction,NaN,0,A&B_SS1,Sol: EIF_Sol - 400 mm,Sol,EIF_Sol - 400 mm,NaN,NaN,1406624,1,"162,177223428329","0,31","0,4",0,"134,96","135,36","0,4","-3,34000000000023","-2,94000000000023","-2,94000000000023","-3,34000000000023",NaN,1,0,NaN,NaN,NaN,0,0,0,<Aucun>,NaN,NaN,NaN,0,0UcXdq41b0rv1UZqd9YDoK,NaN,NaN,NaN,NaN,NaN,EIF_STR - BETON,3,"0,7",604440,"0,382409177820268","2,615",NaN,NaN,0,NaN,"0,4",NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
636929,NaN,NaN,NaN,NaN,EIF_Sol - 230 mm,3,7,"636502,636506,636846,636857,639749,639756,639759",0,NaN,5,23,"1107656,1108070,1108469,1111200,1111442,1111535,1113864,1114165,1115604,1116181,1116183,1118803,1118912,1118966,1129058,1138314,1141144,1141591,1143089,1180899,1180969,1181061,1181234",0,NaN,0,0,NaN,0,NaN,0,False,False,<Aucun>,Sols,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNWw,0,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,Enrobage d'armature 1 <25 mm>,-1,"107,922940125838","469,230174460166",Aucun(e),Nouvelle construction,NaN,0,C_RDC,Sol: EIF_Sol - 230 mm,Sol,EIF_Sol - 230 mm,NaN,NaN,684287,1,"104,846492013679","-0,13","0,23",0,"138,42","138,65","0,23","0,119999999999752","0,349999999999752","0,349999999999752","0,119999999999752",NaN,1,0,NaN,NaN,NaN,0,0,0,<Aucun>,NaN,NaN,NaN,0,0gcOVK$Xn1XgDKvIBnA8vn,NaN,NaN,NaN,NaN,NaN,EIF_STR - BETON,3,"0,7",347553,"0,219885277246654","4,54782608695652",NaN,NaN,0,NaN,"0

In [ ]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Vérifier les noms de colonnes nettoyés
print("Colonnes disponibles dans df_test :", df_test.columns.tolist())


X_test = df_test
y_pred = pipeline.predict(X_test)

# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred, columns=targets)

# Afficher les premières lignes
print(df_pred.head(50))

Colonnes disponibles dans df_test : ['tableau_1']
Colonnes après flatten : ['tableau_1']


KeyError: "None of [Index(['ouvertures', 'epaisseur', 'profil_modifie', 'materiau_structurel',\n       'famille_et_type', 'nom'],\n      dtype='object')] are in the [columns]"

In [ ]:
df_pred.to_csv("resultats_predictionssols.csv", index=False)

In [ ]:
df_pred.head(50)

,011ec_lot,012ec_ouvrage,013ec_localisation,014ec_mode_constructif
0,GO,PLANCHER,COURANT,PREDALLE USINE
1,GO,PLANCHER,COURANT,PREDALLE USINE
2,GO,PLANCHER,COURANT,PREDALLE USINE
3,GO,PLANCHER,COURANT,PREDALLE USINE
4,GO,PLANCHER,COURANT,PREDALLE USINE
5,GO,PLANCHER,COURANT,PREDALLE USINE
6,GO,PLANCHER,COURANT,PREDALLE USINE
7,GO,PLANCHER,COURANT,PREDALLE USINE
8,GO,PLANCHER,COURANT,PREDALLE USINE
9,GO,PLANCHER,COURANT,PREDALLE USINE
